In [14]:
from IPython.display import Markdown as md
from pathlib import Path
md(Path("readme.md").read_text())

This repository contains Python code and a Jupyter Notebook
running the [CONTIN program by S. Provencher](http://dx.doi.org/10.1016/0010-4655(82)90174-6)
on every DLS measurement (`*.ASC` files)
at the specified angle found in the given subfolders.


## TODO
- fix multicore (parallel) calc on Windows
- [done] plot measured and fitted correlation curve
- [done] reviewed units of *ptRange* and *fitRange* CONTIN parameters
  - *fitRange* is given in meters now
- [done] output peak statistics with uncertainties
  - by calculating the statistics of lower and upper distribution (uncertainty subtracted from and added to distribution result) and using the max. absolute value
- [done] float formatting in CONTIN input file fixed
- [done] output CONTIN error message if no output was generated

# Some Parameters (please adjust)

## Specify the measurement folder
(And mind the scattering angle in a cell further down!)

In [ ]:
dataDir = '142 2020 MW002-02'
dataDir = '../20210511/094 2021 PS-Standard 1zu1000'

## CONTIN parameters

In [ ]:
continConfig = dict(
    angle=26, recalc=False,
    ptRangeSec=(3e-7, 1e0), fitRangeM=(1e-9, 500e-9), gridpts=500,
    transformData=True, freeBaseline=True, weighResiduals=False,
)

In [ ]:
from jupyter_analysis_tools.utils import updatedDict
angles = [26, 34, 42, 50, 58, 66, 74, 82, 90, 98, 106, 114, 122, 130, 138, 146]
continConfigs = [updatedDict(continConfig, 'angle', angle)
                 for angle in angles]
#continConfigs

# Process given data directory

## Find data files

In [ ]:
import jupyter_analysis_tools
jupyter_analysis_tools.utils.setPackage(globals())

In [ ]:
from jupyter_analysis_tools.datalocations import getDataDirs, getDataFiles
dataDirs = getDataDirs(dataDir, noWorkDir=True)
#dataFiles = getDataFiles(dataDirs, include="*raged.ASC")#, exclude="_average")
dataFiles = getDataFiles(dataDirs, include="*.ASC")#, exclude="_average")
dataFiles

## Run CONTIN on each file

In [ ]:
from .contin import runContinOverFiles, getContinResults

resultDirs = runContinOverFiles(dataFiles, continConfigs, nthreads=None)
resultDirs

## Fetch CONTIN results

In [ ]:
# show first result for testing
dn = resultDirs[32]
dfDistrib, dfFit, _ = getContinResults(dn)
dfDistrib.plot('radius', 'distrib', yerr='err', ecolor='salmon', grid=True, label=dn.name);

In [ ]:
from jupyter_analysis_tools.distrib import normalizeDistrib, findPeakRanges, findLocalMinima
idx = findPeakRanges(dfDistrib.radius, dfDistrib.distrib, tol=1e-6)
findLocalMinima(idx, dfDistrib.radius.values, dfDistrib.distrib.values, doPlot=True, verbose=True)

## Plot results from all files

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from jupyter_analysis_tools.plotting import plotColor, initFigure, lineWidth
from jupyter_analysis_tools.distrib import distrParFromPeakRanges, area, distrParLatex
from jupyter_analysis_tools.distrib import normalizeDistrib, findPeakRanges
import numpy as np

class GenericResult:
    color = None

    @staticmethod
    def getBarWidth(xvec):
        return np.concatenate((np.diff(xvec)[:1], np.diff(xvec)))

    @classmethod
    def plotPeakRange(cls, ax, peakRange, peakDistrib, fullDistrib, moments, distrPar):
        #print("plotPeakRange", peakRange, moments['area'])
        x, y, u = peakDistrib
        xvec, yvec, uvec = fullDistrib
        mom, momLo, momHi = moments
        dp, dpLo, dpHi = distrPar
        #ax.plot(x, y, 'o', color=cls.color)
        lbl, fmt = [], "{: <7s} {: 9.2g} ±{: 9.2g}"
        for k in "area", "median", "var", "skew", "kurt":
            if k == "median":
                lbl.append(fmt.format("median:", dp[-1], max(abs(dp[-1]-dpLo[-1]), abs(dpHi[-1]-dp[-1]))))
            else:
                lbl.append(fmt.format(k+':', mom[k], max(abs(mom[k]-momLo[k]), abs(momHi[k]-mom[k]))))
        ax.bar(x, y, width=cls.getBarWidth(x), color=cls.color, alpha=0.5, label="\n".join(lbl))
        ax.fill_between(x, np.maximum(0, y-u), y+u,
                        color='red', lw=0, alpha=0.1, label="uncertainties")
        ax.legend(prop=font_manager.FontProperties(family='monospace')); ax.grid(True);

class ContinResult(GenericResult):
    name = "CONTIN"
    xColumn = "radius"; yColumn = "distrib"; uColumn = "err"
    getResults = getContinResults
    color = plotColor(1)

class Results:
    def __init__(self, filename, rtype=None):
        self.rtype = rtype
        self.sampleDir = Path(filename)
        self.angle     = None
        self.dfDistrib, self.dfFit, self.varmap = self.rtype.getResults(self.sampleDir, self.angle)
        if self.dfDistrib is None: return
        self.x, self.y, self.u = normalizeDistrib(
            self.dfDistrib[self.rtype.xColumn].values,
            self.dfDistrib[self.rtype.yColumn].values,
            self.dfDistrib[self.rtype.uColumn].values)
        self.peakRanges = findPeakRanges(self.x, self.y, tol=1e-6)
        # refine the peak ranges containing multiple maxima
        self.peakRanges = findLocalMinima(self.peakRanges, self.x, self.y)

    def plot(self, axes):
        if self.dfDistrib is None: return
        dp, _ = distrParFromPeakRanges(self.x, self.y, self.u, self.peakRanges,
                                       plot={'func': self.rtype.plotPeakRange,
                                             'axes': axes, 'startIdx': 2})
        # plot fitted correlation curve
        ax = axes[0]
        ax.plot(self.dfFit['tau'], self.dfFit['corrIn'],
               color="black", lw=lineWidth()*2, label="measured")
        ax.plot(self.dfFit['tau'], self.dfFit['corrFit'],
               color=self.rtype.color, label="fit")
        ax.legend()
        ax2 = ax.twinx()
        residual = self.dfFit['corrIn']-self.dfFit['corrFit']
        ax2.plot(self.dfFit['tau'], residual, 'k.', alpha=.3, label="residual")
        ax2.set_ylim([-max(abs(residual)),max(abs(residual))])
        # combine legends
        ax2handles, ax2labels = ax2.get_legend_handles_labels()
        axhandles, axlabels = ax.get_legend_handles_labels()
        ax.legend(axhandles+ax2handles, axlabels+ax2labels)
        ax.grid(); ax.set_xscale("log");
        # plot complete distribution as loaded from file
        ax = axes[1]
        lbl = ("from file, " + self.rtype.name
               + area(self.x, self.y, showArea=True)
               +"\n"+distrParLatex(dp[0]))
        ax.fill_between(self.x, self.y,
               #width=GenericResult.getBarWidth(self.x),
               color=self.rtype.color, alpha=0.5, label=lbl)
        #ax.errorbar(self.x, self.y, yerr=self.u, lw=lineWidth()*2, label=lbl)
        ax.fill_between(self.x,
                        np.maximum(0, self.y-self.u),
                        self.y+self.u,
                        color='red', lw=0, alpha=0.1, label="uncertainties")
        ax.legend(); ax.grid(); ax.set_xscale("log");

def plotResult(filename):
    filename = Path(filename)
    # CONTIN results
    cnt = Results(filename, rtype=ContinResult)
    if not hasattr(cnt, 'peakRanges'):
        return # nothing to do
    nsubplots = 2+len(cnt.peakRanges)
    fig, axes = plt.subplots(1, nsubplots, dpi=100, gridspec_kw=dict(wspace=.4))
    initFigure(fig, width=nsubplots*120, aspectRatio=nsubplots/1., quiet=True)
    fig.suptitle("…"+str(filename)[-60:], fontsize=10)

    cnt.plot(axes)
    plt.savefig(filename.with_suffix('.png'))
    return cnt

In [ ]:
results = [plotResult(resultDir) for resultDir in resultDirs]

In [ ]:
result = [res for res in results if "averaged" not in res.sampleDir.name][0]
result.sampleDir, result.varmap

In [ ]:
plt.plot(result.x, result.y)